In [1]:
import json
import random
import warnings
from pathlib import Path

import load_data
import numpy as np
import stylized_score
import train_fourier_flow

## Set seeds for reconstruction of the results

In [2]:
# Set seeds.
SEED = 12345
np.random.seed(SEED)
random.seed(SEED)

#### Load Data

Filter out selected times and stocks

In [3]:
min_len = 9216
B = 500
S = 8
L = 4096
price_df = load_data.load_prices("sp500")
log_ret, symbols = load_data.get_log_returns(price_df, min_len, True)
stf = stylized_score.boostrap_stylized_facts(log_ret, B, S, L)

Timespan 1987-12-17 -- 2024-07-17
Number of symbols 234


In [4]:
# get data
run_dir = Path("../data/fourierflow_runs")
experiments = []
for i, path in enumerate(run_dir.iterdir()):
    if (path / "final/model.pt").exists():
        print(f"Working on {str(path)}, nr {i}")

        try:
            model = train_fourier_flow.load_fourierflow(path / "final/model.pt")
            with (path / "final/model_info.json").open() as file:
                info = json.load(file)

            def sampler(S):
                with warnings.catch_warnings(action="ignore"):
                    return train_fourier_flow.sample_fourierflow(model, S)

            m_stf = stylized_score.stylied_facts_from_model(sampler, B, S)
            tot, ind, _ = stylized_score.stylized_score(stf, m_stf)

            experiments.append(
                {
                    "score": tot,
                    "ind_scores": ind,
                    "path": str(path),
                    "name": path.stem,
                    "dist": info["dist"],
                    "mark": "\\xmark"
                    if len(info["stylized_losses"]) == 0
                    else "\\cmark",
                }
            )
        except:  # noqa E722
            print(f"Failed {str(path)}")

        experiments.sort(key=lambda x: x["score"])
        with (run_dir / "experiments.json").open("w") as file:
            file.write(json.dumps(experiments, ensure_ascii=True, indent=4))

Working on ../data/fourierflow_runs/FourierFlow_2024_08_21-11_02_30, nr 0
Working on ../data/fourierflow_runs/FourierFlow_2024_08_21-11_03_22, nr 1
Working on ../data/fourierflow_runs/FourierFlow_2024_08_21-11_02_58, nr 2
Working on ../data/fourierflow_runs/FourierFlow_2024_08_21-11_03_39, nr 3
Working on ../data/fourierflow_runs/FourierFlow_2024_08_21-11_07_11, nr 4
Working on ../data/fourierflow_runs/FourierFlow_2024_08_21-11_02_39, nr 5


In [5]:
print_experiemnts = []
for exp in experiments:
    print_experiemnts.append(
        (exp["score"], exp["ind_scores"], exp["dist"], exp["mark"])
    )

print_experiemnts = sorted(print_experiemnts, key=lambda x: x[0])

# Print Tables for thesis

In [6]:
table_start = r"""\begin{table}
  \centering
  \begin{threeparttable}
    \caption{Fourier Flow model scores}
    \label{tab:fourierflow}
        \begin{tabular}{l|c|l|l|l|l|l|l|l}
            \toprule
            distribution & stylized losses & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
                         & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\"""
table_end = """            \\bottomrule
        \\end{tabular}
        \\begin{tablenotes}[flushleft]
            \\footnotesize
            \\item Fourier Flow model scores. The model distribution was obtained by resampling 500 times
            eight sequences of lenght 4096 and computing the averaged estimators for the stylized facts
            over the eight samples. The score is obtained by a comparison of the bootstrapped real distribution
            and the model distribution obtained by resampling.
        \\end{tablenotes}
  \\end{threeparttable}
\\end{table}"""

print(table_start)
for mu, scs, model, mark in print_experiemnts:
    ind_scores_str = " & ".join([f"${d:.2f}$" for d in scs])
    print(r"            \hline")
    print(f"            {model} & {mark} & {mu:.2f} & {ind_scores_str}\\\\")

print(table_end)

\begin{table}
  \centering
  \begin{threeparttable}
    \caption{Fourier Flow model scores}
    \label{tab:fourierflow}
        \begin{tabular}{l|c|l|l|l|l|l|l|l}
            \toprule
            distribution & stylized losses & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
                         & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\
            \hline
            studentt & \cmark & 3.07 & $3.41$ & $0.78$ & $1.08$ & $1.30$ & $3.39$ & $8.48$\\
            \hline
            laplace & \xmark & 3.32 & $2.86$ & $0.51$ & $4.81$ & $1.84$ & $1.38$ & $8.50$\\
            \hline
            normal & \xmark & 3.35 & $2.17$ & $0.48$ & $4.71$ & $2.14$ & $2.07$ & $8.51$\\
            \hline
            normal & \cmark & 3.43 & $1.91$ & $0.56$ & $4.72$ & $1.95$ & $2.86$ & $8.59$\\
            \hline
            laplace & \cmark & 3.50 & $3.18$ & $0.25$ & $4.50$ & $2.09$ & $2.60$ & $8.39$\\
            \hline
            studentt & \xmark &